Gradiente Descendente Estocástico (Stochastic Gradient Descent)

- Atualizar os pesos a cada exemplo de treino (leia-se a cada cálculo com uma amostra do conjunto de treino).
- Implementação de um treinamento parcial 'partial-fit' que não reinicializa os pesos, que podemos utilizar para treinamento 'online'.
- Para verificar a convergência após o treinamento, vamos calcular o custo como a média dos custos em cada época.
- Funcionalidade para bagunçar os dados antes de cada época para evitar ciclos repetitivos com o random_state.

In [36]:
import numpy as np

In [ ]:
class AdalineSGD(object):
    """Adaptive Linear Neuron classifier.

    Parameters
    ----------
    eta: float
        Learning rate (bwtween 0.0 and 1.0)
    n_iter: int
        Passes over the training dataset (number of epochs).
    shuffle: bool (default: True)
        Shuffles the training data every epoch if True to prevent cycles.
    random_state: int
        Random number generator seed for weight initialization.
    
    
    Attributes
    ----------
    w_: 1d-array
        Weights after fitting.
    cost_: list
        Sum-of-squares cost function value averaged over all training examples in each epoch.
    
    """

    def __init__(self, eta=0.01, n_iter=10, shuffle=True, random_state=None):
        self.eta = eta
        self.n_iter = n_iter
        self.w_initializated = False
        self.shuffle = shuffle
        self.random_state = random_state

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X: {array-like}, shape = [n_examples, n_features]
            Training vectors, where n_example is the number of examples and n_features is the number of features.
        y: array-like, shape = [n_examples]
            Target values.

        Returns
        -------
        self: object
        
        """
        self._initialized_weights(X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            if self.shuffle:
                X, y = self._shuffle(X, y)
            cost = []

            for xi, target in zip(X, y):
                cost.append(self._update_weights(xi, target))
            avg_cost = sum(cost) / len(y)
            self.cost_.append(avg_cost)
        
        return self
    
    def partial_fit(self, X, y):
        """Fit training data without reinitializing the weights"""
        if not self.w_initializated:
            self._initialize_weights(X.shape[1])
        if y.ravel().shape[0] > 1:
            for xi, target in zip(X, y):
                self._update_weights(xi, target)
        else:
            self._update_weights(X, y)
        return self
    
    def _shuffle(self, X, y):
        """Shuffle training data"""
        r = self.rgen.permutation(len(y))
        return X[r], y[r]
    
    def _initialize_weights(self, m):
        """Initialize weights to small random numbers"""
        self.rgen = np.random.RandomState(self.random_state)
        self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=1+m)
        self.w_initializated = True
    
    def _update_weights(self, xi, target):
        """Apply Adaline learning rule to update the weights"""

